In [28]:
import pandas as pd
from fullcontact import FullContactClient
import numpy as np
from imblearn.over_sampling import SMOTENC
from helpers import Data
from urllib.request import urlopen, Request
import mysql.connector
from getpass import getpass
from mysql.connector import connect
from bs4 import BeautifulSoup
import requests
import re
from helpers import individualFeatures, Data, Scraping


w = pd.read_csv(r'C:\Users\aacjp\Sales-Scientist\datasets\check6.csv')

In [29]:
from helpers import Scraping

In [41]:
string = '''
Courses as low as $13.99
For an extra 10% off sale prices, use SAVE10 in cart. Ends Aug. 26.

What do you want to learn?
What do you want to learn?
A broad selection of courses
Choose from 155,000 online video courses with new additions published every month
Expand your career opportunities with Python
Whether you work in machine learning or finance, or are pursuing a career in web development or data science, Python is one of the most important skills you can learn. Python's simple syntax is especially suited for desktop, web, and business applications. Python's design philosophy emphasizes readability and usability. Python was developed upon the premise that there should be only one way (and preferably one obvious way) to do things, a philosophy that has resulted in a strict level of code standardization. The core programming language is quite small and the standard library is also large. In fact, Python's large library is one of its greatest benefits, providing a variety of different tools for programmers suited for many different tasks.


2021 Complete Python Bootcamp From Zero to Hero in Python
Jose Portilla
Rating: 4.6 out of 5
4.6
(378,451)
Current price$23.99
Original Price$139.99

Machine Learning A-Z™: Hands-On Python & R In Data Science
Kirill Eremenko, Hadelin de Ponteves, SuperDataScience Support, Ligency Team
Rating: 4.5 out of 5
4.5
(149,626)
Current price$13.99
Original Price$89.99
Bestseller

Python for Data Science and Machine Learning Bootcamp
Jose Portilla
Rating: 4.6 out of 5
4.6
(103,723)
Current price$13.99
Original Price$89.99

Learn Python Programming Masterclass
Tim Buchalka, Jean-Paul Roberts, Tim Buchalka's Learn Programming Academy
Rating: 4.5 out of 5
4.5
(79,891)
Current price$22.99
Original Price$139.99

The Python Mega Course: Build 10 Real World Applications
Ardit Sulce
Rating: 4.6 out of 5
4.6
(51,410)
Current price$21.99
Original Price$129.99






Learn in-demand skills with over 155,000 video courses
Choose courses taught by real-world experts
Learn at your own pace, with lifetime access on mobile and desktop
Students are viewing
'''

len(string.split(' '))

277

In [42]:
from helpers import individualFeatures

In [43]:
individualFeatures().triggerWords(string) #string)

1

In [3]:
df = w.iloc[-2:]
df

,name,landingPage,customer,domain,model,source,adspend,hardcosts,revenue,avg_polarity,...,global,percentPaid,sales,adRevenue,cac,roas,profit,profitMargin,lpc,averageOrderValue
57,cleverprogrammer.com,https://www.cleverprogrammer.com/frontend-trai...,0,technology,info,NaN,50000,18000,100000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,udemy.com,https://www.udemy.com/,0,technology,info,NaN,12500000,12000000,33000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
from convert import Features, Wrappers

ImportError: cannot import name 'Scrapers' from 'helpers' (C:\Users\aacjp\Sales-Scientist\helpers.py)

The Hack: http://sangaline.com/post/advanced-web-scraping-tutorial/

In [5]:
connection = connect(host='localhost', user='root', password='Raptor//Kona9', database='sys')
cursor = connection.cursor()

In [6]:
def analyzeTweets(keyword):
    keys = open(r'C:\Users\aacjp\OneDrive\Desktop\ssh\Twitter\keys.txt').readlines()
    a1 = keys[0].replace('\n', '') 
    a2 = keys[1].replace('\n', '') 
    c1 = keys[2].replace('\n', '') 
    c2 = keys[3].replace('\n', '') 
    auth = tweepy.OAuthHandler(c1, c2)
    auth.set_access_token(a1, a2)
    api = tweepy.API(auth)
    topic = api.search(keyword)
    polarity = []
    subjectivity = []
    for i in range(len(topic)):
        tweet = topic[i]._json['text'].replace('@', '')
        blob = TextBlob(tweet)
        sent = blob.sentiment
        polarity.append(sent[0])
        subjectivity.append(sent[1])
    mu_p = np.mean(polarity)
    mu_s = np.mean(subjectivity)
    std_p = np.std(polarity)
    std_s = np.std(subjectivity)
    return mu_p, mu_s, std_p, std_s
    
def getLeads(df):
    return list(df['name'])
    
def leadsOnTwitter(df):
    leads = getLeads(df)
    mup = []
    mus = []
    stdp = []
    stds = []
    counter = 0
    for lead in leads:
        k = lead.split('.')[0]
        mu_p, mu_s, std_p, std_s = Data().analyzeTweets(k)
        mup.append(mu_p)
        mus.append(mu_s)
        stdp.append(std_p)
        stds.append(std_s)
        counter += 1
        print(counter)
    return mup, mus, stdp, stds
    
def globalRank(df):
    leads = getLeads(df)
    key = open(r'C:\Users\aacjp\OneDrive\Desktop\ssh\fullContact\access.txt').read()
    client = FullContactClient(key)
    ranks = []
    counter = 0
    for lead in leads:
        counter += 1
        print(counter)
        req = client.company.enrich(domain=lead)
        try:
            rank = req.json()['details']['traffic']['countryRank']['global']['rank']
        except: 
            rank = None
        ranks.append(rank)
    return ranks    

exceeded request limit, try again later

In [7]:
mup, mus, stdp, stds = leadsOnTwitter(df)

1
2


In [8]:
df['avg_polarity'] = mup
df['avg_subjectivity'] = mus
df['std_polarity'] = stdp
df['std_subjectivity'] = stds

<ipython-input-8-b04a74d34fc7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['avg_polarity'] = mup
<ipython-input-8-b04a74d34fc7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['avg_subjectivity'] = mus
<ipython-input-8-b04a74d34fc7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [9]:
df['global'] = globalRank(df)

1
2


<ipython-input-9-2cfd31d0330c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['global'] = globalRank(df)


In [22]:
df2 = Wrappers().aggregate(df.reset_index())
df2

,index,name,landingPage,customer,domain,model,source,adspend,hardcosts,revenue,...,global,percentPaid,sales,adRevenue,cac,roas,profit,profitMargin,lpc,averageOrderValue
0,57,cleverprogrammer.com,https://www.cleverprogrammer.com/frontend-trai...,0,technology,info,NaN,50000,18000,100000,...,249212,0.4069,NaN,9.936539e+04,NaN,1.987308,32000,0.320000,NaN,NaN
1,58,udemy.com,https://www.udemy.com/,0,technology,info,NaN,12500000,12000000,33000000,...,232,0.3783,NaN,3.302423e+07,NaN,2.641939,8500000,0.257576,NaN,NaN


In [25]:
df3 = pd.concat([w[:-2], df2]).reset_index()
df3.drop(['level_0', 'index'], axis='columns').to_csv(r'C:\Users\aacjp\Sales-Scientist\datasets\check6.csv')